In [1]:
using BenchmarkTools
using Revise

In [2]:
using CSV
using DataFrames
using JSON

using DataIO
using GCTGMT
using Normalization
using Support

using FeatureSetEnrichment

┌ Info: Precompiling FeatureSetEnrichment [d7bef1b9-4643-4ed1-9ceb-ed43dd382ec0]
└ @ Base loading.jl:1317


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15656338930442271228\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15281551830535315427\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15656338930442271228\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
name_to_set_to_element_ = Dict(splitpath(path)[end] => read_gmt(path) for path in read_directory("/Users/kwat/Downloads/gene_sets/"))

;

In [4]:
method_ = Vector{String}()

for method in keys(score_set_new(["a", "b"], [-1.0, 1.0], ["a"]; plot = false))

    push!(method_, string(method, " extreme"), string(method, " area"))

end

n_method = length(method_)

;

The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.


In [5]:
benchmarks_directory_path = "/Users/kwat/Downloads/benchmarks/"

"/Users/kwat/Downloads/benchmarks/"

In [6]:
benchmark_ = Vector{String}()

benchmark_x_method_rows = Vector{Vector{Float64}}()

;

In [ ]:
for benchmark_directory_path in reverse(read_directory(benchmarks_directory_path))

    #
    name = splitpath(benchmark_directory_path)[end]

    println(name)

    #
    element_, score_ =
        eachcol(read_data(joinpath(benchmark_directory_path, "gene_x_score.tsv")))

    json_dict = JSON.parse(open(joinpath(benchmark_directory_path, "gene_sets.json")))

    #
    set_to_element_ = Dict{String, Vector{String}}()

    for gmt_path in json_dict["gene_sets_tested"]

        merge!(set_to_element_, name_to_set_to_element_[splitpath(gmt_path)[end]])

    end

    #
    set_ = sort(collect(keys(set_to_element_)))

    n_set = length(set_)

    println("    ", n_set, " set.")

    #
    set_to_method_to_result = score_set_new(element_, score_, set_to_element_)

    #
    set_x_method = Matrix{Float64}(undef, n_set, n_method)

    for (set_i, (set, method_to_result)) in enumerate(set_to_method_to_result)

        set_x_method_row = []

        for result in values(method_to_result)

            append!(set_x_method_row, result[2:3])

        end

        set_x_method[set_i, :] = set_x_method_row

    end

    #
    for set in json_dict["gene_sets_positive"]

        #
        if in(set, set_)

            println("    ", set)

        else

            println("    ", set, " (missing)")

            continue

        end

        #
        benchmark_x_method_row = Vector{Float64}()

        #
        for (method_i, set_score_) in enumerate(eachcol(set_x_method))

            #
            is_negative_ = set_score_ .< 0

            is_positive_ = 0 .<= set_score_

            #
            set_negative_ = set_[is_negative_]

            set_positive_ = set_[is_positive_]

            #
            negative_ = .-normalize(.-set_score_[is_negative_], "1224") / sum(is_negative_)

            positive_ = normalize(set_score_[is_positive_], "1224") / sum(is_positive_)

            #
            i_negative = findfirst(set_negative_ .== set)

            i_positive = findfirst(set_positive_ .== set)

            if i_negative == nothing && i_positive == nothing

                println("        ", method_[method_i], ": ", set_score_[findfirst(set_ .== set)])

                continue

            elseif i_negative != nothing

                benchmark_score = negative_[i_negative]

            elseif i_positive != nothing

                benchmark_score = positive_[i_positive]

            end

            push!(benchmark_x_method_row, benchmark_score)

        end

        #
        if 0 == length(benchmark_x_method_row)

            println("        Bad gene set.")

        else

            push!(benchmark_, string(name, '.', set))

            push!(benchmark_x_method_rows, benchmark_x_method_row)

        end

    end

end

99
    4762 set.
    KEGG_SPLICEOSOME
    KEGG_RNA_DEGRADATION
    REACTOME_P53_DEPENDENT_G1_DNA_DAMAGE_RESPONSE
    REACTOME_P53_INDEPENDENT_G1_S_DNA_DAMAGE_CHECKPOINT
    REACTOME_REPAIR_SYNTHESIS_FOR_GAP_FILLING_BY_DNA_POL_IN_TC_NER
98
    4811 set.
    FOCAL_ADHESION
    LIANG_SILENCED_BY_METHYLATION (missing)
    MISSIAGLIA_REGULATED_BY_METHYLATION_UP
97
    5001 set.
    CORDENONSI_YAP_CONSERVED_SIGNATURE
    HALLMARK_APICAL_JUNCTION
    HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION
96
    14765 set.
    GO_DNA_REPLICATION
    GO_DNA_DEPENDENT_DNA_REPLICATION
    GO_REGULATION_OF_DNA_DEPENDENT_DNA_REPLICATION
    GO_DNA_STRAND_ELONGATION
    GO_DNA_STRAND_ELONGATION_INVOLVED_IN_DNA_REPLICATION
95
    6111 set.
    AML_TCP_vs_untreated_UP
94
    4811 set.
    Erythroid_associated_genes
93
    4811 set.
    Amino_Acid_Metabolism
92
    5917 set.
    GO_REGULATION_OF_MONONUCLEAR_CELL_MIGRATION
91
    4811 set.
    LSC_ASSOCIATED_GENES
90
    4762 set.
    SCHUHMACHER_MYC_TARGETS_UP
   

In [ ]:
benchmark_x_method = convert_vector_of_vector_to_matrix(benchmark_x_method_rows)

benchmark_x_method = DataFrame(benchmark_x_method, method_)

insertcols!(benchmark_x_method, 1, Symbol("Benchmark") => benchmark_)

In [ ]:
benchmark_x_method_path = "benchmark_x_method.tsv"

;

In [ ]:
CSV.write(benchmark_x_method_path, benchmark_x_method)

;

In [ ]:
benchmark_x_method = read_data(benchmark_x_method_path)

In [ ]:
benchmark_x_method_is = Matrix{}(undef, length(benchmark_), n_method)

for (benchmark_i, benchmark_score_) in enumerate(eachrow(select(benchmark_x_method, Not(Symbol("Benchmark")))))
    
    classic_score = benchmark_score_[Symbol("Is KS < extreme")]
    
    benchmark_score_ = Vector(benchmark_score_)
    
    if classic_score < 0
        
        is_ = benchmark_score_ .< classic_score
        
    else
        
        is_ = classic_score .< benchmark_score_
        
    end
    
    benchmark_x_method_is[benchmark_i, :] = is_
    
end

In [ ]:
for (method, n_better) in zip(sort_like(sum(eachrow(benchmark_x_method_is)), method_)...)
    
    println(method, "    ", n_better)
    
end